In [1]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
using Reel
using POMDPs
using POMDPSimulators
using POMDPPolicies
using POMDPModelTools
using ARDESPOT

In [2]:
include("atan2.jl")
include("obs.jl")
rng = MersenneTwister(2);


In [3]:
# random walk

POS_0 = [6.0, 60.0]
CRS_0 = 90 # target's course
HDG_0 = 90 # o/s heading
SPD_0 = 1 # 1 or 2

#input is course in degrees and rng
#returns next course in degrees
function next_crs(crs,rng)
    if rand(rng) < .9
        return crs
    end
    return (crs + rand(rng,[-1,1])*30 % 360)
end

# state as tuple (x, y, crs, hdg) of target (hdg of o/s)
function f(state, control, rng)
    r, θ, crs, hdg, spd = state
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + cos(π/180*crs) - spd*cos(π/180*hdg), y + sin(π/180*crs) - spd*sin(π/180*hdg)]
    crs = next_crs(crs,rng)
    hdg = control[1]
    spd = control[2]
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = atan2(pos[1],pos[2])*180/π
    if θ < 0 θ += 360 end
    return [r, θ, crs, hdg, spd]
end


# likelihood weight of observation "o" given x, a, xp


x = []
courses = []
observs = []
crs = CRS_0
pos = POS_0
hdg = HDG_0
spd = SPD_0

for i in 1:100
    state = f((pos[1], pos[2], crs, hdg, spd),(0,1),rng)
    pos[1], pos[2], crs, hdg, spd = state
    θ = state[2]
    rad = state[1]
    if θ < 0 θ += 360 end
    push!(courses,crs)
    push!(observs,(rad, θ, obs0(state), obs1(state), obs2(state), obs3(state)))
end


In [ ]:

for i in observs
   #println(i)
end


In [4]:
function r(s,a)
    range = sqrt(s[1]^2 + s[2]^2)
    if range > 100 return -1 end
    if range <= 10 return -100 end
    return 0
end

r (generic function with 1 method)

In [5]:
model = ParticleFilterModel{Vector{Float64}}(f, g)

N = 1000
filter = SIRParticleFilter(model, N);



In [6]:
# implement POMDP here

mutable struct targetPOMDP <: POMDP{Vector{Float64}, Tuple{Float64}, Int64}
    p_walk::Float64
    discount::Float64
end

statespace = [(10*r,θ*15, crs*15, hdg*15, spd) for r in 0:30, θ in 1:12, crs in 1:12, hdg in 1:12, spd in 1:2]

targetPOMDP() = targetPOMDP(0.9, 0.9)
POMDPs.actions(::targetPOMDP) = ((30,1),(0,1),(-30,1),(30, 2), (0,2), (-30,2))
POMDPs.actionindex(::targetPOMDP, a::Tuple) = (a[1]/30+1)*2 + a[2] #cleverness returns 1-6
POMDPs.states(::targetPOMDP) = statespace
POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Int64}) = LinearIndices(statespace)[s[1]/10, s[2]/30, s[3]/30, s[4]/30, s[5]]
#POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Int64}) = LinearIndices(statespace)[round(Int,s[1]/10)+1, round(Int,s[2]/30)+1,
#    (round(Int,s[3]/30)+1, round(Int,s[4]/30)+1, round(Int,s[5])+1]
POMDPs.observations(::targetPOMDP) = (0, 1, 2, 3)
POMDPs.obsindex(::targetPOMDP, o::Int64) = o + 1

POMDPs.initialstate_distribution(::targetPOMDP) = ParticleCollection([[5, 60, 90, 90, 1] for i in 1:N])
POMDPs.initialstate(::targetPOMDP, rng::AbstractRNG) = [5, 60, 90, 90, 1]
POMDPs.isterminal(::targetPOMDP,s) = s[1] >= 300
POMDPs.discount(::targetPOMDP) = 0.9

function POMDPs.gen(m::targetPOMDP, s, a, rng)
    return (sp=f(s,a,rng), r=r(s), o=h(s,rng))
end

In [7]:
N = 1000
pomdp = targetPOMDP(0.9, 0.9)
#initialstate_distribution(pomdp) = ParticleCollection([[5, 60, 90, 90, 1] for i in 1:N])


updater = SIRParticleFilter(pomdp, N);

